# bleacher report API key

In [159]:
import json
import requests
from bs4 import BeautifulSoup
import time
# from newsapi import NewsApiClient # !pip install newsapi-python

In [26]:
API_KEY = "b942a2d2391e4ad8ae01c9168a5cdcc1" # salbro@mit.edu
API_KEY = "90493a497ace4b8ebe8ef5e82bfadce6" # cyrille's API key
API_KEY = "f604134c17ae431591a2976d7c3bae55" # stephenpalbro@gmail.com api key

In [90]:
param_dict = {"from": "2018-03-20", "sources":"bleacher-report",
              "pageSize":"100", "apiKey":API_KEY, "q":"lebron"} # you can also do "sortBy=popularity"

def get_url(param_dict):
    url = "https://newsapi.org/v2/everything?"
    for k, v in param_dict.items():
        url += (k + "=" + v + "&")
    return url

In [86]:
def get_articles(param_dict, page_end = 100, page_start=1):
    url_root = get_url(param_dict)
    responses = [requests.get(url_root + "page="+str(page)).json() for page in range(page_start, page_end + 1)]
    return responses

In [87]:
responses = get_articles(param_dict, page_end = 50)

In [91]:
all_articles = []
for response in responses:
    all_articles += response['articles']

In [160]:
len(all_articles)

265

## Now let's scrape article text

In [181]:
%%time
docs = []
for article in all_articles:
    url = article['url']
    soup = BeautifulSoup(requests.get(url).text, "lxml")
    content = soup.find("div", {"class":"organism contentStream"})
    if content is None:
        content_slides = soup.find_all("div", {"class":"organism contentStream slide"})
        paragraphs = []
        for content_slide in content_slides:
            paragraphs += content_slide.find_all("p")
    else:
        paragraphs = content.find_all("p")
    
    entire_text = "".join([p.get_text() for p in paragraphs])
    docs.append(entire_text)
    time.sleep(.300)

### Now let's scrape the comments

In [185]:
%%time
comments_by_doc = []
for article in all_articles[:10]:
    comments = []
    url = article['url']
    soup = BeautifulSoup(requests.get(url).text, "lxml")
    content = soup.find_all("span", {"class":"_5mdd"})
    if content is None:
        print("no comment with class _5mdd")
    for comment_span in content:
        comments.append(comment_span.get_text())
    comments_by_doc.append(comments)
        


CPU times: user 394 ms, sys: 36.2 ms, total: 430 ms
Wall time: 848 ms


In [209]:
url = all_articles[0]['url']

In [210]:
soup = BeautifulSoup(requests.get(url).text, "lxml")
# content = soup.find_all("iframe", {"title":"Facebook Social Plugin"})
# # content
# content

In [198]:
url = "https://www.facebook.com/plugins/feedback.php?api_key=135174055162&channel_url=https%3A%2F%2Fstaticxx.facebook.com%2Fconnect%2Fxd_arbiter%2Fr%2FFdM1l_dpErI.js%3Fversion%3D42%23cb%3Df2a4b22fc286bc%26domain%3Dbleacherreport.com%26origin%3Dhttps%253A%252F%252Fbleacherreport.com%252Ff2d8cf4393b3b8c%26relation%3Dparent.parent&href=http%3A%2F%2Fbleacherreport.com%2Farticles%2F2768785&locale=en_US&numposts=10&sdk=joey&version=v2.8&width=100%25"

In [219]:
soup.find("footer").find("div", {"class":"fb-comments"}).find("span")


In [220]:
soup.find("div", {"id": "facebook"})

In [221]:
soup("iframe")

[]